# Dimensionality Reduction
Many machine learning problems involve thousands or even millions of features for each training instance. Not only do all these features make training extremely slow, but they can also make it much harder to find a good solution, as we will see. This problem is often referred to as the *curse of dimensionality*.

It is possible to reduce the number of features considerably.

For example, consider the MNIST images: the pixels on the image borders are almost always white, so you could completely drop these pixels fromt the training set without losing much information. Additionally, two neighboring 
pixels are often highly correlated: if you merge them into a single pixel, you will not lose much information.

There are two main approaches to dimensionality reduction - *projection* and *Manifold Learning*.

The three most popular dimensionality reduction techniques are: *PCA, Kenel PCA* and *LLE*.

## The Curse of Dimensionality
![the curse of Dimensionality](images/tcofd.jpg)

## Projection
![Projection 1](images/projection1.jpg)
![Projection 2](images/projection2.jpg)
![Projection 3](images/projection3.jpg)

## Manifold Learning

Swiss roll split in two classes:

3D Space | 2D Space
- | -
![ML 3](images/manifold_decision_boundary_plot3.png) | ![ML 4](images/manifold_decision_boundary_plot4.png)
![ML 1](images/manifold_decision_boundary_plot1.png) | ![ML 2](images/manifold_decision_boundary_plot2.png)

In the first row, (on the left), the decision boundary would be fairly complex, but in the 2D unrolled manifold space(on the right), the decision boundary is a straight line.

In the second row, the decision boundary looks simple in the original 3D space (a vertical plane), but it looks more complex in the unrolled manifold. 

## PCA
Principal Component Analysis (PCA) identifies the hyperplane that lies closest to the data, and then it projects the data onto it.

PCA identifies the axis thata accounts for largest amount of variance in the training set. It also find a second axis, orthogonal to the first one, that accounts for the largest amount of remaining variance.

In [28]:
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))

X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

In [29]:
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]

In [32]:
m, n = X.shape

S = np.zeros(X_centered.shape)
S[:n, :n] = np.diag(s)

In [33]:
np.allclose(X_centered, U.dot(S).dot(Vt))

True

In [34]:
W2 = Vt.T[:, :2]
X2D = X_centered.dot(W2)

In [35]:
X2D_using_svd = X2D

In [36]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)

In [38]:
X2D[:5]

array([[ 1.26203346,  0.42067648],
       [-0.08001485, -0.35272239],
       [ 1.17545763,  0.36085729],
       [ 0.89305601, -0.30862856],
       [ 0.73016287, -0.25404049]])

In [40]:
X2D_using_svd[:5]

array([[-1.26203346, -0.42067648],
       [ 0.08001485,  0.35272239],
       [-1.17545763, -0.36085729],
       [-0.89305601,  0.30862856],
       [-0.73016287,  0.25404049]])

### PCA for Compression

Applying PCA to the MNIST dataset while preserving 95% its variance.

In [41]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.target = mnist.target.astype(np.uint8)

In [42]:
from sklearn.model_selection import train_test_split

X = mnist["data"]
y = mnist["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [43]:
pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1 # 95% variance

d

154

In [50]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

In [51]:
pca.n_components_

154

In [52]:
np.sum(pca.explained_variance_ratio_)

0.9503684424557437

After compression, each instance is left with 154 features, instead of the original 784 features. So while most of the dataset is preserved, the dataset is now less than 20% of its original size!

It is also possible to decompress the reduced dataset back to 784 dimensions by applying the inverse transformation of the PCA projection. This will give you back the original data (with 5% variance that was dropped).

In [48]:
pca = PCA(n_components=154)

# compress data to 154 dimensions
X_reduced = pca.fit_transform(X_train)

X_reduced.shape

(52500, 154)

In [49]:
# decompress data back to original 784 dimensions
X_recovered = pca.inverse_transform(X_reduced)

X_recovered.shape

(52500, 784)

### Randomized PCA

By setting *svd_solver* hyperparameter to "randomized", Scikit-learn uses a stochastic algorithm called *Randomized PCA* that quickly finds an approximation of first *d* principal components.
Its computational complexity is O(m x d$^2$) + O(d$^3$) instead of  O(m x n$^2$) + O(n$^3$), so it is dramatically faster than *full* SVD when d is much smaller than m:

In [54]:
rnd_pca = PCA(n_components=154, svd_solver='randomized')

X_reduced = rnd_pca.fit_transform(X_train)

(52500, 154)

By default *svd_solver* is set to "auto": Scikit-learn automatically uses the randomized PCA algorithm if *m* or *n* is greater than 500 and *d* is less than 80% of *m* or *n*, or else it uses the *full* SVD approach.

### Incremental PCA

The above implementations of PCA require whole training set to be loaded in the memory. Fortunately, Incremental PCA 
allows to split the training set in min-batches and feed an IPCA algorithm one mini-batch at a time.

This is useful for large training sets and for applying PCA online (i.e on the fly, as new instances arrive).

In [55]:
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)

for X_batch in np.array_split(X_train, n_batches):
    inc_pca.partial_fit(X_batch)
    
X_reduced = inc_pca.transform(X_train)

### Kernel PCA
A kernel trick is a mathematical technique that implicitly maps instances into a very high-dimensional space (called the feature space).

In [ ]:
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components=2, kernel='rbf', gamma=0.04)

X_reduced = rbf_pca.fit_transform(X)